# Feathr Quick Start Notebook

This notebook illustrates the use of Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

The major problems Feathr solves are:

1. Create, share and manage useful features from raw source data.
2. Provide Point-in-time feature join to create training dataset to ensure no data leakage.
3. Deploy the same feature data to online store to eliminate training and inference data skew.

## Prerequisite

Feathr has native cloud integration. First step is to provision required cloud resources if you want to use Feathr.

Follow the [Feathr ARM deployment guide](https://feathr-ai.github.io/feathr/how-to-guides/azure-deployment-arm.html) to run Feathr on Azure. This allows you to quickly get started with automated deployment using Azure Resource Manager template. For more details, please refer [README.md](https://github.com/feathr-ai/feathr#%EF%B8%8F-running-feathr-on-cloud-with-a-few-simple-steps).

Additionally, to run this notebook, you'll need to install `feathr` pip package. For local spark, simply run `pip install feathr` on the machine that runs this notebook. To use Databricks or Azure Synapse Analytics, please see dependency management documents:
- [Azure Databricks dependency management](https://learn.microsoft.com/en-us/azure/databricks/libraries/)
- [Azure Synapse Analytics dependency management](https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-azure-portal-add-libraries)

## Notebook Steps

This tutorial demonstrates the key capabilities of Feathr, including:

1. Install Feathr and necessary dependencies
2. Create shareable features with Feathr feature definition configs
3. Create training data using point-in-time correct feature join
4. Train a prediction model and evaluate the model and features
5. Register the features to share across teams
6. Materialize feature values for online scoring

The overall data flow is as follows:

<img src="https://raw.githubusercontent.com/feathr-ai/feathr/main/docs/images/feature_flow.png" width="800">

## 1. Install Feathr and Necessary Dependancies

Install feathr and necessary packages by running one of following commends if you haven't installed them already:

In [12]:
# To install feathr from the latest codes in the repo:
#%pip install "git+https://github.com/feathr-ai/feathr.git#subdirectory=feathr_project&egg=feathr[notebook]" 

# To install the latest release:
#%pip install "feathr[notebook]"

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from datetime import timedelta
import os
from pathlib import Path

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.sql import DataFrame, SparkSession
import pyspark.sql.functions as F

import feathr
from feathr import (
    FeathrClient,
    # Feature data types
    BOOLEAN, FLOAT, INT32, ValueType,
    # Feature data sources
    INPUT_CONTEXT, HdfsSource,
    # Feature aggregations
    TypedKey, WindowAggTransformation,
    # Feature types and anchor
    DerivedFeature, Feature, FeatureAnchor,
    # Materialization
    BackfillTime, MaterializationSettings, RedisSink,
    # Offline feature computation
    FeatureQuery, ObservationSettings,
)
from feathr.datasets import nyc_taxi
from feathr.spark_provider.feathr_configurations import SparkExecutionConfiguration
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df
from feathr.utils.platform import is_databricks, is_jupyter

print(f"Feathr version: {feathr.__version__}")

Feathr version: 1.0.0


## 2. Create Shareable Features with Feathr Feature Definition Configs

First, we define all the necessary resource key values for authentication. These values are retrieved by using [Azure Key Vault](https://azure.microsoft.com/en-us/services/key-vault/) cloud key value store. For authentication, we use Azure CLI credential in this notebook, but you may add secrets' list and get permission for the necessary service principal instead of running `az login --use-device-code`.

Please refer to [A note on using azure key vault to store credentials](https://github.com/feathr-ai/feathr/blob/41e7496b38c43af6d7f8f1de842f657b27840f6d/docs/how-to-guides/feathr-configuration-and-env.md#a-note-on-using-azure-key-vault-to-store-credentials) for more details.

In [15]:
RESOURCE_PREFIX = None  # TODO fill the value used to deploy the resources via ARM template
PROJECT_NAME = "nyc_taxi"

# Currently support: 'azure_synapse', 'databricks', and 'local' 
SPARK_CLUSTER = "local"

# TODO fill values to use databricks cluster:
DATABRICKS_CLUSTER_ID = None             # Set Databricks cluster id to use an existing cluster
if is_databricks():
    # If this notebook is running on Databricks, its context can be used to retrieve token and instance URL
    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    DATABRICKS_WORKSPACE_TOKEN_VALUE = ctx.apiToken().get()
    SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL = f"https://{ctx.tags().get('browserHostName').get()}"
else:
    DATABRICKS_WORKSPACE_TOKEN_VALUE = None                  # Set Databricks workspace token to use databricks
    SPARK_CONFIG__DATABRICKS__WORKSPACE_INSTANCE_URL = None  # Set Databricks workspace url to use databricks

# TODO fill values to use Azure Synapse cluster:
AZURE_SYNAPSE_SPARK_POOL = None  # Set Azure Synapse Spark pool name
AZURE_SYNAPSE_URL = None         # Set Azure Synapse workspace url to use Azure Synapse
ADLS_KEY = None                  # Set Azure Data Lake Storage key to use Azure Synapse

# An existing Feathr config file path. If None, we'll generate a new config based on the constants in this cell.
FEATHR_CONFIG_PATH = None

# If set True, use an interactive browser authentication to get the redis password.
USE_CLI_AUTH = False

# If set True, register the features to Feathr registry.
REGISTER_FEATURES = False

# (For the notebook test pipeline) If true, use ScrapBook package to collect the results.
SCRAP_RESULTS = False

To use Databricks as the feathr client's target platform, you may need to set a databricks token to an environment variable like:

`export DATABRICKS_WORKSPACE_TOKEN_VALUE=your-token`

or in the notebook cell,

`os.environ["DATABRICKS_WORKSPACE_TOKEN_VALUE"] = your-token`

If you are running this notebook on Databricks, the token will be automatically retrieved by using the current Databricks notebook context.

On the other hand, to use Azure Synapse cluster, you have to specify the synapse workspace storage key:

`export ADLS_KEY=your-key`

or in the notebook cell,

`os.environ["ADLS_KEY"] = your-key`

In [16]:
if SPARK_CLUSTER == "azure_synapse" and not os.environ.get("ADLS_KEY"):
    os.environ["ADLS_KEY"] = ADLS_KEY
elif SPARK_CLUSTER == "databricks" and not os.environ.get("DATABRICKS_WORKSPACE_TOKEN_VALUE"):
    os.environ["DATABRICKS_WORKSPACE_TOKEN_VALUE"] = DATABRICKS_WORKSPACE_TOKEN_VALUE

In [17]:
# Get an authentication credential to access Azure resources and register features
if USE_CLI_AUTH:
    # Use AZ CLI interactive browser authentication
    !az login --use-device-code
    from azure.identity import AzureCliCredential
    credential = AzureCliCredential(additionally_allowed_tenants=['*'],)
elif "AZURE_TENANT_ID" in os.environ and "AZURE_CLIENT_ID" in os.environ and "AZURE_CLIENT_SECRET" in os.environ:
    # Use Environment variable secret
    from azure.identity import EnvironmentCredential
    credential = EnvironmentCredential()
else:
    # Try to use the default credential
    from azure.identity import DefaultAzureCredential
    credential = DefaultAzureCredential(
        exclude_interactive_browser_credential=False,
        additionally_allowed_tenants=['*'],
    )

### Configurations

Feathr uses a yaml file to define configurations. Please refer to [feathr_config.yaml]( https://github.com//feathr-ai/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) for the meaning of each field.

All the Feathr configurations can be set to the yaml file via keyword arguments of `generate_config` helper function. Each keyword argument should be the concatenation of different layers of the config name using `__` as a separator.
For example, if you want to specify a different value for the feature registry api endpoint, you can pass `        feature_registry__api_endpoint="YOUR-API-ENDPOINT-URL"`.

Note, a default value for the api endpoint will be set based on `RESOURCE_PREFIX`.

In [18]:
os.environ['JDBC_USER'] = "root"
os.environ['JDBC_PASSWORD'] = "DsteamIC2024"
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"
os.environ['REDIS_PASSWORD'] = "foobared"  # default password for Redis


# Make sure we get the Feathr jar name, assuming we just have one jar file.
PROJECT_NAME = "nyc_taxi"

All the configurations can be overwritten by environment variables with concatenation of `__` for different layers of the config file, same as how you may pass the keyword arguments to `generate_config` utility function.

For example, `feathr_runtime_location` for databricks config can be overwritten by setting `spark_config__databricks__feathr_runtime_location` environment variable.

### Initialize Feathr client

In [19]:
feathr_workspace_folder = Path(f"./{PROJECT_NAME}_feathr_config.yaml")
client = FeathrClient(str(feathr_workspace_folder))

2024-09-06 16:26:16.893 | INFO     | feathr.utils._env_config_reader:get:62 - Config secrets__azure_key_vault__name is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 16:26:16.896 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__s3__s3_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 16:26:16.897 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__adls__adls_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 16:26:16.898 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__wasb__wasb_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 16:26:16

### Prepare the NYC taxi fare dataset

In [20]:
# If the notebook is runnong on Jupyter, start a spark session:
if is_jupyter():
    spark = (
        SparkSession
        .builder
        .appName("feathr")
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.0,io.delta:delta-core_2.12:2.1.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.ui.port", "8080")  # Set ui port other than the default one (4040) so that feathr spark job doesn't fail. 
        .getOrCreate()
    )

# Else, you must already have a spark session object available in databricks or synapse notebooks.

In [21]:
# Use dbfs if the notebook is running on Databricks
if is_databricks():
    WORKING_DIR = f"/dbfs/{PROJECT_NAME}"
else:
    WORKING_DIR = PROJECT_NAME

In [22]:
# Download the data file
data_file_path = "../../feathr_project/test/test_user_workspace/green_tripdata_2020-04_with_index.csv"
df_raw = nyc_taxi.get_spark_df(spark=spark, local_cache_path=data_file_path)
df_raw.limit(5).show()

+-------+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|trip_id|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+-------+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|      0|     2.0| 2020-04-01 00:44:02|  2020-04-01 00:52:23|                 N|       1.0|          42|          41|            1.0|      

In [23]:
TIMESTAMP_COL = "lpep_dropoff_datetime"
TIMESTAMP_FORMAT = "yyyy-MM-dd HH:mm:ss"

# Get features from register server

In [24]:
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, return_keys=True, verbose=True)

2024-09-06 16:26:19.656 | INFO     | feathr.client:get_features_from_registry:1147 - Get anchor features from registry: 
2024-09-06 16:26:19.657 | INFO     | feathr.client:get_features_from_registry:1153 - {
  "name": "f_trip_distance",
  "featureType": {
    "type": "TENSOR",
    "tensorCategory": "DENSE",
    "dimensionType": [],
    "valType": "FLOAT"
  },
  "key": [
    {
      "keyColumn": "NOT_NEEDED",
      "keyColumnType": "UNSPECIFIED",
      "fullName": "feathr.dummy_typedkey",
      "description": "feathr.dummy_typedkey",
      "keyColumnAlias": "NOT_NEEDED"
    }
  ],
  "transformation": {
    "transformExpr": "trip_distance"
  }
}
2024-09-06 16:26:19.658 | INFO     | feathr.client:get_features_from_registry:1153 - {
  "name": "f_trip_time_duration",
  "featureType": {
    "type": "TENSOR",
    "tensorCategory": "DENSE",
    "dimensionType": [],
    "valType": "FLOAT"
  },
  "key": [
    {
      "keyColumn": "NOT_NEEDED",
      "keyColumnType": "UNSPECIFIED",
      "fullNam

In [25]:
[feat.name for feat in list(feature_dict[0].values())]

['f_trip_distance',
 'f_trip_time_duration',
 'f_is_long_trip_distance',
 'f_day_of_week',
 'f_day_of_month',
 'f_hour_of_day',
 'f_location_avg_fare',
 'f_location_max_fare',
 'f_trip_speed']

In [26]:
[type_key.key_column for type_keys in list(feature_dict[1].values()) for type_key in type_keys]

['NOT_NEEDED',
 'NOT_NEEDED',
 'NOT_NEEDED',
 'NOT_NEEDED',
 'NOT_NEEDED',
 'NOT_NEEDED',
 'DOLocationID',
 'DOLocationID',
 'NOT_NEEDED']

## 6. Materialize Feature Values for Online Scoring

While we computed feature values on-the-fly at request time via Feathr, we can pre-compute the feature values and materialize them to offline or online storages such as Redis.

Note, only the features anchored to offline data source can be materialized.

In [27]:
# Get the last date from the dataset
backfill_timestamp = (
    df_raw
    .select(F.to_timestamp(F.col(TIMESTAMP_COL), TIMESTAMP_FORMAT).alias(TIMESTAMP_COL))
    .agg({TIMESTAMP_COL: "max"})
    .collect()[0][0]
)
backfill_timestamp

datetime.datetime(2020, 5, 1, 0, 47)

In [28]:
FEATURE_TABLE_NAME = "nycTaxiDemoFeature"

# Time range to materialize
backfill_time = BackfillTime(
    start=backfill_timestamp,
    end=backfill_timestamp,
    step=timedelta(days=1),
)

# Destinations:
# For online store,
redis_sink = RedisSink(table_name=FEATURE_TABLE_NAME)

# For offline store,
# adls_sink = HdfsSink(output_path=)

settings = MaterializationSettings(
    name=FEATURE_TABLE_NAME + ".job",  # job name
    backfill_time=backfill_time,
    sinks=[redis_sink],  # or adls_sink
    feature_names=['f_location_avg_fare',
                     'f_location_max_fare',],
)

client.materialize_features(
    settings=settings,
    execution_configurations={"spark.feathr.outputFormat": "parquet"},
)

client.wait_job_to_finish(timeout_sec=500)

2024-09-06 16:26:20.423 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config KAFKA_SASL_JAAS_CONFIG is not found in the environment variable or the remote key value store.
2024-09-06 16:26:20.424 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__url is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 16:26:20.424 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__user is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 16:26:20.425 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config MONITORING_DATABASE_SQL_PASSWORD is not found in the environment variable or the remote key value store.
2024-09-06 16:26:20.425 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Loca

>

	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.2 in central
	found com.typesafe#config;1.3.4 in central
	found org.apache.hadoop#hadoop-mapreduce-client-core;3.3.2 in central
	found org.apache.hadoop#hadoop-yarn-client;3.3.2 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found commons-cli#commons-cli;1.2 in central
	found log4j#log4j;1.2.17 in central
	found org.eclipse.jetty.websocket#websocket-client;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-client;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-http;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-util;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-io;9.4.43.v20210629 in central
	found org.eclipse.jetty.websocket#websocket-common;9.4.43.v20210629 in central
	found org.eclipse.jetty.websocket#websocket-api;9.4.43.v20210629 in central
	found org.apache.hado

x

	found com.sun.jersey.contribs#jersey-guice;1.19 in central
	found com.sun.jersey#jersey-servlet;1.19 in central
	found com.fasterxml.jackson.module#jackson-module-jaxb-annotations;2.13.0 in central
	found jakarta.xml.bind#jakarta.xml.bind-api;2.3.3 in central
	found jakarta.activation#jakarta.activation-api;1.2.1 in central
	found com.fasterxml.jackson.jaxrs#jackson-jaxrs-json-provider;2.13.0 in central
	found com.fasterxml.jackson.jaxrs#jackson-jaxrs-base;2.13.0 in central
	found org.slf4j#slf4j-log4j12;1.7.30 in central
	found org.jline#jline;3.9.0 in central
	found io.netty#netty;3.10.6.Final in central
	found org.apache.hadoop#hadoop-common;3.3.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found commons-net#commons-net;3.6 in central
	found commons-collections#commons-collections;3.2.2 in central
	found org.eclipse.jetty#jetty-server;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-servlet;9.4.43.v20210629 in central
	found org.eclipse.jetty#jett

>

2024-09-06 16:26:54.470 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:234 - Spark job with pid 110517 finished in: 34 seconds.


>

Now, you can retrieve features for online scoring as follows:

In [29]:
# Note, to get a single key, you may use client.get_online_features instead
materialized_feature_values = client.multi_get_online_features(
    feature_table=FEATURE_TABLE_NAME,
    keys=["239", "265"],
    feature_names=['f_location_avg_fare',
                     'f_location_max_fare',],
)
materialized_feature_values

{'239': [1480.53271484375, 5707.0], '265': [4160.6171875, 10000.0]}

## Cleanup

In [30]:
# TODO: Unregister, delete cached files or do any other cleanups.

In [31]:
# Stop the spark session if it is a local session.
if is_jupyter():
    spark.stop()